# Golden Dataset & RAG Benchmarking (Jupyter)

Bu notebook, bir RAG sisteminin **ölçülebilir** şekilde iyileştirilmesi için gereken akışı gösterir:

1) **Golden dataset** (soru + ground-truth cevap + beklenen kaynaklar)  
2) **Chunking** (baseline vs iyileştirilmiş)  
3) **Retrieval benchmarking**: Recall@k, MRR, nDCG  
4) (Opsiyonel) **Faithfulness / Answer Relevance** (LLM-as-a-judge veya kural-tabanlı proxy)



## 0) Kurulum

- Bu demo **LLM zorunlu değil**: retrieval metrikleri tamamen offline ölçülür.
- LLM tabanlı metrikler (faithfulness/relevance) **opsiyonel** ve notebook içinde yorum satırında.


In [82]:
%pip -q install pandas numpy scikit-learn rank-bm25 sentence-transformers faiss-cpu google-cloud-aiplatform google-genai pydantic


# (Opsiyonel) RAGAS:
# %pip -q install ragas

# Kurulum sonrası kernel restart gerekebilir.



[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1) Mini Corpus (Demo)



In [83]:
import pandas as pd

pd.set_option('max_colwidth', 400)



docs = [
    {
        "doc_id": "D1",
        "title": "Vertex AI Studio",
        "text": (
            "Vertex AI Studio, prompt denemeleri ve model testleri için kullanılır. "
            "Language, Vision ve Speech gibi modlarda çalışabilir. "
            "Parametreler: temperature, top-p, max_output_tokens."
        ),
    },
    {
        "doc_id": "D2",
        "title": "Document AI Form Extractor",
        "text": (
            "Document AI, PDF gibi belgelerden yapılandırılmış veri çıkarır. "
            "Form Extractor key-value çiftlerini yakalamaya odaklanır. "
            "Invoice Processor gibi specialized processorlar entities döndürebilir. "
            "HITL (Human-in-the-loop) doğruluğu artırmak için kullanılır."
        ),
    },
    {
        "doc_id": "D3",
        "title": "RAG Optimizasyonu",
        "text": (
            "RAG kalitesini ölçmek için retrieval metrikleri: Recall@k, MRR, nDCG. "
            "Chunking stratejileri (recursive, semantic) sonucu etkiler. "
            "Hybrid search: BM25 + vektör arama. Re-ranking en alakalı sonuçları yukarı taşır."
        ),
    },
]

df_docs = pd.DataFrame(docs)
df_docs


,doc_id,title,text
0,D1,Vertex AI Studio,"Vertex AI Studio, prompt denemeleri ve model testleri için kullanılır. Language, Vision ve Speech gibi modlarda çalışabilir. Parametreler: temperature, top-p, max_output_tokens."
1,D2,Document AI Form Extractor,"Document AI, PDF gibi belgelerden yapılandırılmış veri çıkarır. Form Extractor key-value çiftlerini yakalamaya odaklanır. Invoice Processor gibi specialized processorlar entities döndürebilir. HITL (Human-in-the-loop) doğruluğu artırmak için kullanılır."
2,D3,RAG Optimizasyonu,"RAG kalitesini ölçmek için retrieval metrikleri: Recall@k, MRR, nDCG. Chunking stratejileri (recursive, semantic) sonucu etkiler. Hybrid search: BM25 + vektör arama. Re-ranking en alakalı sonuçları yukarı taşır."


## 2) Chunking

İki basit chunker:
- **Baseline**: sabit uzunluk (char bazlı)
- **Improved**: cümle tabanlı (basit semantic-ish)

Gerçek projede: doc-type bazlı splitter / recursive / semantic splitter kullanılır.


In [84]:
import re
from typing import List, Dict

def chunk_fixed(text: str, chunk_size: int = 200, overlap: int = 30) -> List[str]:
    chunks = []
    i = 0
    while i < len(text):
        chunks.append(text[i:i+chunk_size].strip())
        i += max(1, chunk_size - overlap)
    return [c for c in chunks if c]

def chunk_sentences(text: str, max_chars: int = 240) -> List[str]:
    sents = [s.strip() for s in re.split(r"(?<=[.!?])\s+", text) if s.strip()]
    chunks, buf = [], ""
    for s in sents:
        if len(buf) + len(s) + 1 <= max_chars:
            buf = (buf + " " + s).strip()
        else:
            if buf:
                chunks.append(buf)
            buf = s
    if buf:
        chunks.append(buf)
    return chunks

def build_chunks(docs: List[Dict], mode: str = "baseline") -> pd.DataFrame:
    rows = []
    for d in docs:
        chunks = chunk_fixed(d["text"]) if mode == "baseline" else chunk_sentences(d["text"])
        for j, ch in enumerate(chunks):
            rows.append({
                "doc_id": d["doc_id"],
                "title": d["title"],
                "chunk_id": f"{d['doc_id']}_C{j+1}",
                "text": ch
            })
    return pd.DataFrame(rows)

df_chunks_baseline = build_chunks(docs, mode="baseline")
df_chunks_improved = build_chunks(docs, mode="improved")

print("baseline chunks:", len(df_chunks_baseline))
print("improved chunks:", len(df_chunks_improved))
df_chunks_improved.head()


baseline chunks: 6
improved chunks: 4


,doc_id,title,chunk_id,text
0,D1,Vertex AI Studio,D1_C1,"Vertex AI Studio, prompt denemeleri ve model testleri için kullanılır. Language, Vision ve Speech gibi modlarda çalışabilir. Parametreler: temperature, top-p, max_output_tokens."
1,D2,Document AI Form Extractor,D2_C1,"Document AI, PDF gibi belgelerden yapılandırılmış veri çıkarır. Form Extractor key-value çiftlerini yakalamaya odaklanır. Invoice Processor gibi specialized processorlar entities döndürebilir."
2,D2,Document AI Form Extractor,D2_C2,HITL (Human-in-the-loop) doğruluğu artırmak için kullanılır.
3,D3,RAG Optimizasyonu,D3_C1,"RAG kalitesini ölçmek için retrieval metrikleri: Recall@k, MRR, nDCG. Chunking stratejileri (recursive, semantic) sonucu etkiler. Hybrid search: BM25 + vektör arama. Re-ranking en alakalı sonuçları yukarı taşır."


## 3) Sparse Retrieval (BM25) — Offline

In [85]:
import numpy as np
from rank_bm25 import BM25Okapi

def tokenize(text: str) -> List[str]:
    return re.findall(r"[\wğüşöçıİĞÜŞÖÇ]+", text.lower())

def bm25_index(chunks_df: pd.DataFrame):
    corpus_tokens = [tokenize(t) for t in chunks_df["text"].tolist()]
    bm25 = BM25Okapi(corpus_tokens)
    return bm25

def bm25_search(bm25, chunks_df: pd.DataFrame, query: str, k: int = 5):
    scores = bm25.get_scores(tokenize(query))
    top_idx = np.argsort(scores)[::-1][:k]
    out = chunks_df.iloc[top_idx].copy()
    out["score"] = scores[top_idx]
    return out.reset_index(drop=True)

bm25_b = bm25_index(df_chunks_baseline)
bm25_i = bm25_index(df_chunks_improved)

bm25_search(bm25_i, df_chunks_improved, "RAG kalitesini ölçmek için hangi retrieval metrikleri kullanılır?", k=5)


,doc_id,title,chunk_id,text,score
0,D3,RAG Optimizasyonu,D3_C1,"RAG kalitesini ölçmek için retrieval metrikleri: Recall@k, MRR, nDCG. Chunking stratejileri (recursive, semantic) sonucu etkiler. Hybrid search: BM25 + vektör arama. Re-ranking en alakalı sonuçları yukarı taşır.",3.782560
1,D2,Document AI Form Extractor,D2_C2,HITL (Human-in-the-loop) doğruluğu artırmak için kullanılır.,0.263349
2,D1,Vertex AI Studio,D1_C1,"Vertex AI Studio, prompt denemeleri ve model testleri için kullanılır. Language, Vision ve Speech gibi modlarda çalışabilir. Parametreler: temperature, top-p, max_output_tokens.",0.190119
3,D2,Document AI Form Extractor,D2_C1,"Document AI, PDF gibi belgelerden yapılandırılmış veri çıkarır. Form Extractor key-value çiftlerini yakalamaya odaklanır. Invoice Processor gibi specialized processorlar entities döndürebilir.",0.000000


## 4) (Opsiyonel) Dense Retrieval

Daha gerçekçi RAG için sentence-transformers + cosine similarity kullanılır.


In [86]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

def dense_index(chunks_df: pd.DataFrame):
    X = embedder.encode(chunks_df["text"].tolist(), normalize_embeddings=True)
    return X

def dense_search(X, chunks_df: pd.DataFrame, query: str, k: int = 5):
    q = embedder.encode([query], normalize_embeddings=True)
    sims = cosine_similarity(q, X)[0]
    top_idx = np.argsort(sims)[::-1][:k]
    out = chunks_df.iloc[top_idx].copy()
    out["score"] = sims[top_idx]
    return out.reset_index(drop=True)

X_i = dense_index(df_chunks_improved)
dense_i = dense_search(X_i, df_chunks_improved, "RAG kalitesini ölçmek için hangi retrieval metrikleri kullanılır?", k=5)
display(dense_i)

XLMRobertaModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


,doc_id,title,chunk_id,text,score
0,D3,RAG Optimizasyonu,D3_C1,"RAG kalitesini ölçmek için retrieval metrikleri: Recall@k, MRR, nDCG. Chunking stratejileri (recursive, semantic) sonucu etkiler. Hybrid search: BM25 + vektör arama. Re-ranking en alakalı sonuçları yukarı taşır.",0.713404
1,D2,Document AI Form Extractor,D2_C2,HITL (Human-in-the-loop) doğruluğu artırmak için kullanılır.,0.446019
2,D2,Document AI Form Extractor,D2_C1,"Document AI, PDF gibi belgelerden yapılandırılmış veri çıkarır. Form Extractor key-value çiftlerini yakalamaya odaklanır. Invoice Processor gibi specialized processorlar entities döndürebilir.",0.303907
3,D1,Vertex AI Studio,D1_C1,"Vertex AI Studio, prompt denemeleri ve model testleri için kullanılır. Language, Vision ve Speech gibi modlarda çalışabilir. Parametreler: temperature, top-p, max_output_tokens.",0.301792


## 5) Golden Dataset

Önerilen alanlar:
- `query`: soru
- `answer`: ground-truth kısa cevap (insan doğrulamalı)
- `relevant_doc_ids`: doğru doküman(lar) veya chunk id


In [87]:
golden = [
    {
        "qid": "Q1",
        "query": "RAG kalitesini ölçmek için hangi retrieval metrikleri kullanılır?",
        "answer": "Recall@k, MRR ve nDCG gibi retrieval metrikleri kullanılır.",
        "relevant_doc_ids": ["D3"],
    },
    {
        "qid": "Q2",
        "query": "Document AI'da HITL ne işe yarar?",
        "answer": "Human-in-the-loop (HITL), insan onayıyla doğruluğu artırmak için kullanılır.",
        "relevant_doc_ids": ["D2"],
    },
    {
        "qid": "Q3",
        "query": "Vertex AI Studio hangi amaçla kullanılır ve hangi parametreler test edilir?",
        "answer": "Prompt denemeleri/model testleri için; temperature, top-p, max_output_tokens gibi parametreler test edilir.",
        "relevant_doc_ids": ["D1"],
    },
]

df_golden = pd.DataFrame(golden)
df_golden


,qid,query,answer,relevant_doc_ids
0,Q1,RAG kalitesini ölçmek için hangi retrieval metrikleri kullanılır?,"Recall@k, MRR ve nDCG gibi retrieval metrikleri kullanılır.",[D3]
1,Q2,Document AI'da HITL ne işe yarar?,"Human-in-the-loop (HITL), insan onayıyla doğruluğu artırmak için kullanılır.",[D2]
2,Q3,Vertex AI Studio hangi amaçla kullanılır ve hangi parametreler test edilir?,"Prompt denemeleri/model testleri için; temperature, top-p, max_output_tokens gibi parametreler test edilir.",[D1]


## 6) Retrieval Benchmarking — Recall@k, MRR, nDCG

Demo: doc_id üstünden ölçüyoruz.


In [ ]:
import math
from typing import Set

def dcg(rels: List[int]) -> float:
    return sum((2**rel - 1) / math.log2(i+2) for i, rel in enumerate(rels))

def ndcg_at_k(retrieved_doc_ids: List[str], relevant: Set[str], k: int) -> float:
    rels = [1 if d in relevant else 0 for d in retrieved_doc_ids[:k]]
    ideal = sorted(rels, reverse=True)
    denom = dcg(ideal)
    return dcg(rels) / denom if denom > 0 else 0.0

def mrr_at_k(retrieved_doc_ids: List[str], relevant: Set[str], k: int) -> float:
    for i, d in enumerate(retrieved_doc_ids[:k]):
        if d in relevant:
            return 1.0 / (i+1)
    return 0.0

def recall_at_k(retrieved_doc_ids: List[str], relevant: Set[str], k: int) -> float:
    got = set(retrieved_doc_ids[:k]) & relevant
    return len(got) / len(relevant) if relevant else 0.0

def eval_retrieval(chunks_df: pd.DataFrame, bm25, golden_df: pd.DataFrame, k: int = 5) -> pd.DataFrame:
    rows = []
    for _, g in golden_df.iterrows():
        res = bm25_search(bm25, chunks_df, g["query"], k=k)
        retrieved_docs = res["doc_id"].tolist()
        relevant = set(g["relevant_doc_ids"])
        rows.append({
            "qid": g["qid"],
            "recall@k": recall_at_k(retrieved_docs, relevant, k),
            "mrr@k": mrr_at_k(retrieved_docs, relevant, k),
            "ndcg@k": ndcg_at_k(retrieved_docs, relevant, k),
            "top_docs": retrieved_docs[:k],
        })
    return pd.DataFrame(rows)

report_baseline = eval_retrieval(df_chunks_baseline, bm25_b, df_golden, k=5)
report_improved = eval_retrieval(df_chunks_improved, bm25_i, df_golden, k=5)

print("Baseline:")
display(report_baseline)

print("Improved:")
display(report_improved)

summary = pd.DataFrame([
    {"setup": "baseline_bm25", "recall@5": report_baseline["recall@k"].mean(), "mrr@5": report_baseline["mrr@k"].mean(), "ndcg@5": report_baseline["ndcg@k"].mean()},
    {"setup": "improved_bm25", "recall@5": report_improved["recall@k"].mean(), "mrr@5": report_improved["mrr@k"].mean(), "ndcg@5": report_improved["ndcg@k"].mean()},
])
summary


## 7) Debug View (Hata Analizi)

“Kaçırdığımız” soruları tek tek incelemek en büyük hızlandırıcı.


In [90]:
def debug_one(qid: str, chunks_df: pd.DataFrame, bm25, golden_df: pd.DataFrame, k: int = 5):
    g = golden_df[golden_df["qid"] == qid].iloc[0]
    res = bm25_search(bm25, chunks_df, g["query"], k=k)
    print("Q:", g["query"])
    print("GT docs:", g["relevant_doc_ids"])
    return res[["doc_id","title","chunk_id","score","text"]]

debug_one("Q3", df_chunks_improved, bm25_i, df_golden, k=5)


Q: Vertex AI Studio hangi amaçla kullanılır ve hangi parametreler test edilir?
GT docs: ['D1']


,doc_id,title,chunk_id,score,text
0,D1,Vertex AI Studio,D1_C1,3.624454,"Vertex AI Studio, prompt denemeleri ve model testleri için kullanılır. Language, Vision ve Speech gibi modlarda çalışabilir. Parametreler: temperature, top-p, max_output_tokens."
1,D3,RAG Optimizasyonu,D3_C1,0.000000,"RAG kalitesini ölçmek için retrieval metrikleri: Recall@k, MRR, nDCG. Chunking stratejileri (recursive, semantic) sonucu etkiler. Hybrid search: BM25 + vektör arama. Re-ranking en alakalı sonuçları yukarı taşır."
2,D2,Document AI Form Extractor,D2_C2,0.000000,HITL (Human-in-the-loop) doğruluğu artırmak için kullanılır.
3,D2,Document AI Form Extractor,D2_C1,0.000000,"Document AI, PDF gibi belgelerden yapılandırılmış veri çıkarır. Form Extractor key-value çiftlerini yakalamaya odaklanır. Invoice Processor gibi specialized processorlar entities döndürebilir."


## 8) LLM-as-a-Judge (Gemini / Vertex AI) — Faithfulness & Answer Relevance (Opsiyonel)

Bu bölüm **opsiyonel** ve ücret doğurur.

Amaç:
- **Answer Relevance**: Cevap, soruyu gerçekten yanıtlıyor mu?
- **Faithfulness**: Cevaptaki iddialar, verilen **context** tarafından destekleniyor mu? (grounding)

Çıktı: her metrik için 0.0–1.0 arası skor + kısa gerekçe.


In [91]:
# LLM Judge konfig
project_id  = "vertextraining-486212"
region      = "europe-west2"
judge_model = "gemini-2.5-flash"

judge_temperature = 0.0
judge_max_output_tokens = 1024




In [92]:
from vertexai import init
from vertexai.generative_models import GenerativeModel, GenerationConfig, SafetySetting, HarmBlockThreshold, HarmCategory
from google import genai

init(project=project_id, location=region)

client = genai.Client(
    vertexai=True,
    project=project_id,
    location=region,
)

print("✅ Judge model hazır:", judge_model, "| region:", region)


✅ Judge model hazır: gemini-2.5-flash | region: europe-west2


## Judge Fonksiyonları

In [93]:
from pydantic import BaseModel, Field

class JudgeOut(BaseModel):
    score: float = Field(..., ge=0.0, le=1.0)
    rationale: str = Field(..., max_length=1024)


def call_judge(prompt: str, *, model=judge_model) -> dict:
    resp = client.models.generate_content(
        model=model,
        contents=prompt,
        config={
            "temperature": 0.0,
            "top_p": 1.0,
            #"max_output_tokens": judge_max_output_tokens,
            # Structured output:
            "response_mime_type": "application/json",
            "response_schema": JudgeOut,
            # Safety settings (isteğe göre gevşet):
            "safety_settings": [
               {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_ONLY_HIGH"},
               {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_ONLY_HIGH"},
               {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_ONLY_HIGH"},
               {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_ONLY_HIGH"},
         ],
        },
    )

    cand = (getattr(resp, "candidates", None) or [None])[0]
    finish = getattr(cand, "finish_reason", None)

    parsed = getattr(resp, "parsed", None)
    if parsed is not None:
        return parsed.model_dump() if hasattr(parsed, "model_dump") else dict(parsed)

    # Buraya düştüyse: muhtemelen blok/boş
    return {
        "score": 0.0,
        "rationale": f"blocked_or_empty (finish_reason={finish})",
    }



def judge_relevance_tr(question: str, answer: str) -> dict:
    prompt = f"""
Sen bir değerlendirme asistanısın.
Görev: Aşağıdaki Soru ve Cevap için "Cevap Uygunluğu" puanı ver.

Puanlama:
- 1.0: Cevap soruyu doğrudan ve yeterince yanıtlıyor.
- 0.5: Kısmen yanıtlıyor; eksik/muğlak.
- 0.0: Alakasız; soruyu yanıtlamıyor.

Soru: {question}
Cevap: {answer}

Sadece JSON döndür.
"""
    return call_judge(prompt)

def judge_faithfulness_tr(question: str, answer: str, contexts: list[str]) -> dict:
    ctx = "\n\n".join([f"[C{i+1}] {c}" for i, c in enumerate(contexts)])
    prompt = f"""
Sen bir değerlendirme asistanısın.
Görev: Cevabın, aşağıdaki Bağlam tarafından ne ölçüde desteklendiğini puanla.

Puanlama:
- 1.0: Cevabın ana noktaları bağlamda açıkça destekleniyor.
- 0.5: Kısmen destekleniyor; bazı kısımlar belirsiz/eksik.
- 0.0: Büyük ölçüde bağlamda yok veya bağlamla uyuşmuyor.

Kural: Yalnızca verilen bağlama dayan.

Soru: {question}

Bağlam:
{ctx}

Cevap: {answer}

Sadece JSON döndür.
"""
    return call_judge(prompt)


### RAG cevaplarını bağlama

In [94]:
# Kendi sistem çıktını buraya koy (qid -> answer)
rag_answers = {
    "Q1": "RAG kalitesini ölçmek için retrieval metrikleri: Recall@k, MRR, nDCG.",
    "Q2": "HITL (Human-in-the-loop) doğruluğu artırmak için kullanılır.",
    "Q3": "Vertex AI Studio, prompt denemeleri ve model testleri için kullanılır.",
}

# Demo için: eksikse golden answer'ı kullan
use_golden_if_missing = True


### Judge evaluate

In [95]:
k_ctx = 3  # judge'a verilecek context parçası sayısı (top-k)

rows = []
for _, g in df_golden.iterrows():
    qid = g["qid"]
    q = g["query"]

    top = bm25_search(bm25_i, df_chunks_improved, q, k=k_ctx)
    contexts = top["text"].tolist()

    ans = rag_answers.get(qid)
    if (ans is None or not str(ans).strip()) and use_golden_if_missing:
        ans = g["answer"]

    rel = judge_relevance_tr(q, ans)
    fai = judge_faithfulness_tr(q, ans, contexts)

    rows.append({
        "qid": qid,
        "answer_relevance": rel["score"],
        "relevance_rationale": rel["rationale"],
        "faithfulness": fai["score"],
        "faithfulness_rationale": fai["rationale"],
        "ctx_doc_ids": top["doc_id"].tolist(),
    })

df_judge = pd.DataFrame(rows)
df_judge


,qid,answer_relevance,relevance_rationale,faithfulness,faithfulness_rationale,ctx_doc_ids
0,Q1,1.0,"Cevap, RAG kalitesini ölçmek için kullanılan temel retrieval metriklerini (Recall@k, MRR, nDCG) doğrudan ve doğru bir şekilde listelemiştir. Soruya tam ve yeterli bir yanıt vermektedir.",1.0,"Cevap, RAG kalitesini ölçmek için kullanılan retrieval metriklerini (Recall@k, MRR, nDCG) doğrudan ve eksiksiz bir şekilde Bağlam [C1]'den almaktadır. Bağlam, cevabın ana noktalarını açıkça desteklemektedir.","[D3, D2, D1]"
1,Q2,1.0,"Cevap, HITL'nin Document AI'daki temel amacını (doğruluğu artırma) doğrudan ve doğru bir şekilde belirtmektedir. Soruyu yeterince yanıtlamaktadır.",0.5,"Cevap, HITL'nin doğruluğu artırmak için kullanıldığı bilgisini doğrudan Bağlam [C1]'den almaktadır. Ancak, soru HITL'nin 'Document AI'da' ne işe yaradığını sormaktadır. Bağlam, Document AI'dan [C2] bahsetse de, HITL'nin bu özel bağlamda kullanıldığına dair açık bir bağlantı kurmamaktadır. Bu nedenle, cevabın ana noktası desteklenirken, sorunun spesifik kapsamı (Document AI) bağlamda tam olarak...","[D2, D2, D3]"
2,Q3,0.5,"Cevap, Vertex AI Studio'nun amacını (prompt denemeleri ve model testleri) doğru bir şekilde belirtiyor ancak hangi parametrelerin test edildiği sorusunu yanıtlamıyor. Bu nedenle kısmen doğru ve eksiktir.",0.5,"Cevap, Vertex AI Studio'nun amacını (prompt denemeleri ve model testleri) Bağlam [C1]'den doğru bir şekilde belirtmektedir. Ancak, sorunun ikinci kısmı olan 'hangi parametreler test edilir?' sorusunu yanıtlamamıştır, oysa bu bilgi de Bağlam [C1]'de mevcuttur (temperature, top-p, max_output_tokens).","[D1, D3, D2]"


In [96]:
df_judge[["answer_relevance","faithfulness"]].mean().to_frame("mean_score")

,mean_score
answer_relevance,0.833333
faithfulness,0.666667
